In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
# Some generic helper functions which I could not find in Julia or OSCAR

# Determine the pivots by computing the RREF and then looking for the first nonzero entry of each row
function pivots(M)

    (r, R) = rref(M)
    pivots = []

    for i in 1:nrows(R)
        for j in 1:ncols(R)

            if R[i,j] != 0
                append!(pivots, [[i,j]])
                break
            end
        end
    end

    return pivots
end


# given a vector of monomials mons and a row vector M of polynomials finds a matrix coeffs such that mons*coeffs = M
# this replaces the function coefficients which we use in macaulay2
function mons_and_coeffs(mons, M)

    coeffs = [[coeff(f, mons[j]) for f in M] for j in 1:length(mons)]

    return matrix(QQ, coeffs)
end

mons_and_coeffs (generic function with 1 method)

In [5]:
# compute the homogeneity space of an ideal and return it in row reduced echelon form
function homogeneity_space(I)

    F = gens(I)
    n = length(gens(parent(F[1])))
    homogeneity_eqs = matrix(ZZ, [[0 for i in 1:n]])

    for f in F

        exps = [exponent_vector(f,i) for i in 1:length(f)]
        eqs = matrix(ZZ, vcat([exps[1]-exps[i] for i in 2:length(exps)]))
        homogeneity_eqs = vcat(homogeneity_eqs, eqs)
    end
    
    return rref(transpose(nullspace(homogeneity_eqs)[2]))[2]
end

# finds the maximal grading of the kernel of a polynomial map phi
# by computing the homogeneity space of the elimination ideal
function max_grading(phi)

    codom = codomain(phi)
    dom = domain(phi)
    elim_ring, z = polynomial_ring(QQ, vcat([string(i) for i in symbols(codom)], [string(i) for i in symbols(dom)]))

    lift_codom = hom(codom, elim_ring, gens(elim_ring)[1:ngens(codom)])
    lift_dom = hom(dom, elim_ring, gens(elim_ring)[ngens(codom)+1:ngens(elim_ring)])

    return homogeneity_space(ideal([lift_dom(x) - lift_codom(phi(x)) for x in gens(dom)]))
end

max_grading (generic function with 1 method)

In [3]:
# computes the basis for domain in degree deg but removes all monomials 
# which correspond to previously computed relations G
function find_basis_in_degree(G, domain, deg) 

    if length(G) == 0
        
        return monomial_basis(domain, deg)
    end

    
    G_shifts = reduce(vcat, [[g*b for b in monomial_basis(domain, deg - degree(g))] for g in G])
    mons = unique!(reduce(vcat, [collect(monomials(f)) for f in G_shifts]))
    coeffs = mons_and_coeffs(mons, G_shifts)
    bad_monomials = [mons[p[1]] for p in pivots(coeffs)]

    return [m for m in monomial_basis(domain, deg) if !(m in bad_monomials)]
end

find_basis_in_degree (generic function with 1 method)

In [6]:
# Example: A product of segree products
R, (x11, x12, x21, x22, y11, y12, y21, y22) = QQ["x11", "x12", "x21", "x22", "y11", "y12", "y21", "y22"]
S, (s1, s2, t1, t2, m1, m2, n1, n2) = QQ["s1", "s2", "t1", "t2", "m1", "m2", "n1", "n2"]
phi = hom(R, S, [s1*t1, s1*t2, s2*t1, s2*t2, m1*n1, m1*n2, m2*n1, m2*n2])
A = max_grading(phi)[:, 9:16]
R, (x11, x12, x21, x22, y11, y12, y21, y22) = grade(R, A)
f1 = x11*x22 - x12*x21
f2 = y11*y22 - y12*y21
G = [f1,  f2];
grA = grading_group(R)
deg = grA([2,1,1,2,2,2,2,2])

graded by [2 1 1 2 2 2 2 2]